SrNo, RollNo, Gender, Name,SubCode Marks, Grade

1, "2345467", "M", "abcd","301", 89, "B1"

2, "2345468", "F", "AAAA","301", 99, "A1"

3, "2345467", "M", "abcd","48", 96, "A2"

4, "2345468", "F", "AAAA","48", 86, "B1"

In [1]:
import pickle
import getpass
import os
import csv
from operator import itemgetter

result1 = "result.dat"
temp = "temp.dat"
path = "result12.csv"
subcodes = "subcode.csv"

In [2]:
#UPLOAD RECORDS TO PROGRAMME
#RUN IT ONCE (RIGHT NOW ALL BINARY FILES ARE EMPTY)
record=[]
absent = []
with open(path,"r") as f:  # copying all records in record list
  data = csv.reader(f)
  for row in data:
    record.append(row)
codes = []
with open(subcodes,"r") as f: # copying all subject codes in codes list
  data = csv.reader(f)
  for row in data:
    codes.append(row)

lst= list(filter(lambda c: c[4] =="" , record))  #Removing records which have missing information
for i in lst:
  if i in record:
    record.remove(i)  

for i in record:
  result={}                 #Updating the record binary file and absent list
  if i[0].isnumeric():
    if i[5].isnumeric():
      with open(result1,"ab") as f:
        result[i[0]]=[i[1],i[2],i[3],i[4],int(i[5]),i[6]]
        pickle.dump(result,f)
        result={}
    else:
      absent.append(i)



In [3]:
#ALL THE FUNCTIONS USED IN THE PROGRAMME ARE IN THIS CELL!!

def invalid():
  print("INVALID INPUT")
  print("TRY AGAIN")
  print(" ")

def updated_rec():
  rec=[]                   # <--THIS WILL CONVERT ALL DICTIONARY REOCRDS IN A NESTED LIST RECORD
  with open(result1,"rb") as f:  #<--DICTIONARY IS USED ONLY IN DELETE AND UPDATE...THIS IS MAINLY FOR REPORTS PART, SO THAT CHANGES MADE ARE VISIBLE IN REPORTS(IF EFFECTIVE)
    flag=1
    while True:
      try:
        content = pickle.load(f)
        for i in content:
          val = content.values()
          for j in val:
            j.insert(0,i)
          rec.append(j)
      except:
        flag=0
      if flag==0:
        break
  return rec
def search():
  rec=updated_rec()
  while True:
    roll = input("Roll number to be searched: ")
    if roll.isnumeric():
      list301 = list(filter(lambda c: c[1] == roll , rec))
      if len(list301)==0:
        print("Roll not found")
        break
      else:
        for i in list301:
          print(i)
        break
    else:
      invalid()
def roll_input_valid():
  while True:
    RNO = input("Roll Number: ")
    rec=updated_rec()
    for i in rec:
      if i[1]==RNO:
        return RNO
  print("Roll number not found")

def subcode_input_valid():
  while True:
    SNO = input("Subcode: ")
    for i in codes:
      if i[0]==SNO:
        return SNO
  print("Subcode not found")

def marks_input_valid():        #<-- FOR GENERAL VALDATION OF MARKS(IS STRING ENTERED BY USER, NUMBER ONLY OR NOT)
  while True:
    marks = input("Marks: ")
    if marks.isnumeric():
      marks = int(marks)
      if marks>=0 and marks<=100:
          return marks
      else:
        invalid()
    else:
       invalid()
def grade_input_valid():         #<-- FOR GENERAL VALIDATION OF GRADES(CORRECTLY ENTERED BY USER OR NOT)
  while True:
    grade = input("Grade: ")
    grade = grade.upper()
    if grade=="E":
      return grade
    elif len(grade) ==2 and (grade[1] in ["1","2"]) and (grade[0]>="A" and grade[0]<="D"):
        return grade
    else:
      invalid()

def add_record():
  result={}
                                           #FOR ADDITION OF INFORMATION
  with open(result1,"rb") as f:
      data = pickle.load(f)
      lst = data.keys()
      SrNo = str(len(lst)+1)

  while True:
      Roll = input("Roll No.: ")                     #INPUT OF ROLL NUMBER
      if Roll.isnumeric():
          break
      else:
        invalid()

  while True:
    Gender = input("Gender: ")                       #INPUT OF GENDER
    Gender = Gender.upper()
    if Gender in ["M","F"]:
      break
    else:
      print("Type 'M' for Male and 'F' for Female")

  while True:
    flag = 0
    Name = input("Name: ")
    Name = Name.upper()                              #INPUT OF NAME
    for i in Name:
      if not((i>="A" and i<="Z") or (i==" ")):
        flag=1
    if flag == 1:
      invalid()
    else: 
      break

  while True:
      subcode = subcode_input_valid()
      if subcode!=None:
        break
      else:
        print("Subcode not found")                    #INPUT OF SUBCODE


  marks = marks_input_valid()                        #INPUT OF MARKS
  grade = grade_input_valid()


  result[SrNo] = [Roll,Gender,Name,subcode,marks,grade]  #ADDITION TO DICTIONARY
  with open(result1,"ab") as f:
    pickle.dump(result,f)     # DUMPING THIS NEW RECORD TO BINARY FILE
  rec=updated_rec()   # UPDATING THE NEW RECORD IN THE LIST REC


def update():
  with open(result1,"rb+") as f:
    FLAG=0
    rno = roll_input_valid()
    sno = subcode_input_valid()
    try:
      while True:
        c_pos = f.tell()
        result =pickle.load(f)
        lst1 = result.values()
        for i in lst1:
          if rno == i[0] and sno==i[3]:
            print("1.Marks")
            print("2.Grade")
            print("3.Back to Main Menu")
            ch1=input('choice: ')
            if ch1=="1":                            # FOR UPDATING MARKS
              N_marks = marks_input_valid()
              i[4] = N_marks
              f.seek(c_pos)
              pickle.dump(result,f)
              FLAG=1
            elif ch1=="2":                          # FOR UPDATING GRADES
              N_grade = grade_input_valid()
              i[5] = N_grade
              f.seek(c_pos)
              pickle.dump(result,f)
              FLAG=1
            elif ch1=="3":
              FLAG=2
        if FLAG==1:
          print("Changes Saved")
          rec=updated_rec()
          break
        elif FLAG==2:
          break
    except:
        if FLAG==0:
          invalid()

def delete():
    f = open(result1,"rb")
    f1 = open(temp,"wb")
    roll = roll_input_valid()
    try:
      while True:
        data = pickle.load(f)
        lst = data.values()
        for i in lst:
          if i[0]!=roll:
            pickle.dump(data,f1)
    except:
      f.close()
      f1.close()
    print("Are you sure that you want to delete all records of roll number",roll,"? (YES/NO)")   #CONFIRMING DELETE RECORD
    while True:
      sure = input()
      sure = sure.lower()
      if sure=="yes":
        os.remove(result1)
        os.rename(temp,result1)
        rec=updated_rec()
        break
      elif sure=="no":
        break
      else:
        print("Type YES or NO")

def reports():
  while True:
    print("1.Highest marks")
    print("2.Students appeared(Males and Females)")
    print("3.Number of students appeared in each subject")
    print("4.Back to main menu")
    ch2 = input("choice: ")

    if ch2=="1":
      marks = 0
      absent=[]
      rec=updated_rec()       #NOW HERE REC IS CALLED FOR REPORTS
      for CODE in codes:
        print(" ")
        print(" ")
        print(CODE[1],CODE[0])
        list301 = list(filter(lambda c: c[4] ==CODE[0] , rec))
        list301 = sorted(list301,key=itemgetter(5),reverse=True)
        for i in list301:
          if i in absent:
            list301.remove(i)
        if list301!=[]:
          marks = list301[0][5]
          for i in list301:
            if i[5]==marks:
              print(i[3], '\t\t', marks)

    elif ch2=="2":
      Male = []
      Female = []
      rec=updated_rec()     #NOW HERE REC IS CALLED FOR REPORTS
      for i in rec:
        if i[2]=="M":
          Male.append(i)
        elif i[2]=="F":
          Female.append(i)
      Male_name= []
      Female_name = []
      for i in Male:
        if i[3] not in Male_name:
            Male_name.append(i[3])
      print(Male_name)
      print("NUMBER OF MALE STUDENTS APPEARED =",len(Male_name))
      for i in Female:
        if i[3] not in Female_name :
            Female_name.append(i[3])
      print(Female_name)
      print("NUMBER OF FEMALE STUDENTS APPREARED =",len(Female_name))

    elif ch2=="3":
      #SUBJECT WISE ANALYSIS
      for CODE in codes:
        print(" ")
        list301 = list(filter(lambda c: c[4] ==CODE[0] , rec))
        print("STUDENTS APPEARED IN",CODE[1],"=",len(list301))

    elif ch2=="4":
      break

    else:
      invalid()

In [4]:
# MAIN PROGRAMME
PWD = getpass.getpass("Password: ")
if PWD == "1":
  while True:
      print("a.Display")
      print("b.Search")
      print("c.Add")
      print("d.Update")
      print("e.Delete")
      print("f.Reports")
      print("g.Exit")
      ch=input("Enter a choice: ")
      ch = ch.lower()
      if ch=="a":
        with open(result1,"rb") as f:
          try:
            while True:
              print(pickle.load(f))
          except:
            print()
        rec = updated_rec()
      elif ch=="b":
        search()
      elif ch=="c":
        add_record()
      elif ch =='d':
        update()
      elif ch=="e":
        delete()
      elif ch=="f":
        reports()
      elif ch=="g":
        print("Thank You!!")
        break
      else:
        invalid()

a.Display
b.Search
c.Add
d.Update
e.Delete
f.Reports
g.Exit
{'1373': ['26606637', 'M', 'NAV ARORA', '48', 83, 'B2']}
{'1372': ['26606636', 'M', 'KARTIKEY CHAUHAN', '48', 77, 'C1']}
{'1371': ['26606635', 'F', 'ZAKRA FATIMA', '48', 75, 'C1']}
{'1370': ['26606634', 'F', 'PRIYANKA KHATUA', '49', 68, 'D2']}
{'1369': ['26606633', 'F', 'BULBUL GUPTA', '48', 93, 'A2']}
{'1368': ['26606632', 'F', 'SIMRAN GULIA', '48', 74, 'C1']}
{'1367': ['26606631', 'F', 'SANYA DOKHORIA', '48', 75, 'C1']}
{'1365': ['26606629', 'F', 'PRAKRITI MEHTA', '49', 66, 'D2']}
{'1364': ['26606628', 'F', 'POOJA CHADHA', '48', 69, 'C2']}
{'1363': ['26606627', 'F', 'MEHAK ARORA', '34', 82, 'D1']}
{'1362': ['26606626', 'F', 'KHUSHBOO ANEJA', '48', 56, 'D2']}
{'1361': ['26606625', 'M', 'AYUSH TAYAL', '34', 85, 'D1']}
{'1359': ['26606623', 'M', 'SHREY BHARDWAJ', '48', 69, 'C2']}
{'1358': ['26606622', 'F', 'SAKSHI PRAJAPATI', '48', 71, 'C2']}
{'1356': ['26606620', 'M', 'RAJAT SAINI', '48', 92, 'A2']}
{'1355': ['26606619', 'F', 

In [5]:
#SUBJECT WISE ANALYSIS
for CODE in codes:
  print(" ")
  list301 = list(filter(lambda c: c[4] ==CODE[0] , rec))
  print("STUDENTS APPEARED IN",CODE[1],"=",len(list301))

 
STUDENTS APPEARED IN ENGLISH = 229
 
STUDENTS APPEARED IN HINDI = 5
 
STUDENTS APPEARED IN HISTORY = 34
 
STUDENTS APPEARED IN POLITICAL SCIENCE = 35
 
STUDENTS APPEARED IN GEOGRAPHY = 35
 
STUDENTS APPEARED IN ECONOMICS = 130
 
STUDENTS APPEARED IN CARNATIC MUSIC (VOCAL) = 6
 
STUDENTS APPEARED IN PSYCHOLOGY = 0
 
STUDENTS APPEARED IN SOCIOLOGY = 32
 
STUDENTS APPEARED IN MATHEMATICS = 146
 
STUDENTS APPEARED IN PHYSICS = 68
 
STUDENTS APPEARED IN CHEMISTRY = 68
 
STUDENTS APPEARED IN BIOLOGY = 23
 
STUDENTS APPEARED IN PHYSICAL EDUCATION = 114
 
STUDENTS APPEARED IN PAINTING = 8
 
STUDENTS APPEARED IN BUSINESS STUDIES = 125
 
STUDENTS APPEARED IN ACCOUNTANCY = 125
 
STUDENTS APPEARED IN HOME SCIENCE = 21
 
STUDENTS APPEARED IN INFORMATICS PRACTICES (NEW) = 23
 
STUDENTS APPEARED IN COMPUTER SCIENCE (NEW) = 0
 
STUDENTS APPEARED IN LEGAL STUDIES = 11


In [7]:
lst=[]
with open(result1,"rb") as f:
  while True:
    try:
      data = pickle.load(f)
      lst.append(data)
    except:
      break
print(len(lst))

1279
